### Составление единой таблицы

In [17]:
import pandas as pd

загрузка датасетов и создание единой таблицы

In [7]:
sales = pd.read_csv('./data/sales_train_validation.csv')
calendar = pd.read_csv('./data/calendar.csv')
prices = pd.read_csv('./data/sell_prices.csv')

In [15]:
id_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
d_cols = [col for col in sales.columns if col.startswith('d_')]

sales_long = sales.melt(
    id_vars=id_cols,
    value_vars=d_cols,
    var_name = 'd',
    value_name='sales'
)

data = sales_long.merge(calendar, on='d', how='left')
data = data.merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

data['date'] = pd.to_datetime(data['date'])
data['sales'] = data['sales'].astype('int32')

assert data["date"].isna().sum() == 0

print("NaN sell_price:", data["sell_price"].isna().mean())

data.head()

NaN sell_price: 0.2108686367994991


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


### Feature engineering

In [18]:
import numpy as np

In [ ]:
def add_features(
    data,
    group_cols = ['store_id', 'item_id'],
    target_col = 'sales',
    price_col = 'sell_price',
    date_col = 'date',
    lags = [1, 7, 14, 28],
    rolling_windows = [7, 14, 28],
    ewm_spans = [7, 28],
    add_price_features = True,
    add_calendar_features = True,
    add_event_features = True
):
    
    data = data.copy()
    data = data.sort_values(group_cols + [date_col]).reset_index(drop=True)